In [1]:
# direct copy and paste from .py files:

In [2]:
MLFLOW_URI = "https://mlflow.lewagon.ai/"
EXPERIMENT_NAME = "UK LON lwb_smr SOLAR train 01" # template
EXPERIMENT_TAGS = {
    'USER': 'hsth',
    'RUN NAME': 'vertex2, operational, please',
    'VERSION': 'M2_R02_15',
    'DESCRIPTION': 'Model VGG16 UNet, 15 epochs, 72k images',
    'LOSS': 'binary_crossentropy',
    'METRICS': 'accuracy, binaryIoU, AUC'
}


UNET_INPUT_SHAPE = (224,224,3)
TEST_BATCH_SIZE = 8
BATCH_SIZE = TEST_BATCH_SIZE
EPOCHS = 2
LOSS='binary_crossentropy'

# testing / otherwise
SAMPLE_ONLY = 1

# output filenames
CHECKPOINT_NAME = 'test_220610_sample_CHECKER_checkpoint' #.h5 added below
CHECKPOINT_PATH = '../raw_data/checkpoints/'

MODEL_NAME = 'test_220610_sample_CHECKER_model_save'  #.h5 added below
MODEL_PATH = '../raw_data/models/'

# CRITICAL
root_path = '../../raw_data'



# TENSOR SLICE DATA LOADER

In [31]:
import os
import pandas as pd
import random #?

In [32]:
random.random_state = 42

In [34]:

x_path = root_path + '/train_RGB_tiles_jpeg/'
x_images = os.listdir(x_path)
y_path = root_path + '/train_mask_tiles_jpeg/'
y_masks = os.listdir(y_path)
        
folders = ['train_RGB_tiles_jpeg', 'train_mask_tiles_jpeg']
folder_path = [f'{root_path}/{folder}' for folder in folders]
folder_path

train_images, train_mask = [], []
if SAMPLE_ONLY == 1:
    for i, filename in enumerate(os.listdir(folder_path[0])):
        if i > (TEST_BATCH_SIZE * 2): break
        train_images.append(f'{root_path}/train_mask_tiles_jpeg/{filename}')
    for i, filename in enumerate(os.listdir(folder_path[1])):
        if i > (TEST_BATCH_SIZE * 2): break
        train_mask.append(f'{root_path}/train_mask_tiles_jpeg/{filename}')
elif SAMPLE_ONLY == 0: 
    train_images =[f'{root_path}/train_RGB_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[0])]
    train_mask = [f'{root_path}/train_mask_tiles_jpeg/{filename}' for filename in os.listdir(folder_path[1])]

    
train_images.sort()
train_mask.sort()


train_df = pd.DataFrame()
# train_df['file_path'] = train_images
train_df['image_path'] = train_images
train_df['mask_path'] = train_mask

train_df.head()

,image_path,mask_path
0,../../raw_data/train_mask_tiles_jpeg/austin19_...,../../raw_data/train_mask_tiles_jpeg/austin26_...
1,../../raw_data/train_mask_tiles_jpeg/austin31_...,../../raw_data/train_mask_tiles_jpeg/austin33_...
2,../../raw_data/train_mask_tiles_jpeg/chicago11...,../../raw_data/train_mask_tiles_jpeg/austin35_...
3,../../raw_data/train_mask_tiles_jpeg/chicago11...,../../raw_data/train_mask_tiles_jpeg/chicago29...
4,../../raw_data/train_mask_tiles_jpeg/chicago16...,../../raw_data/train_mask_tiles_jpeg/chicago30...


In [45]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

def holdout(df, train_ratio=0.8, test_to_val_ratio=0.5, include_all=False):

    img_paths = df["image_path"].values
    msk_paths = df["mask_path"].values

    df_mask = df.copy()

    df_train, df_val = train_test_split(df_mask, train_size=train_ratio)
    df_test, df_val = train_test_split(df_val, test_size=test_to_val_ratio)

    ds_train = tf.data.Dataset.from_tensor_slices(
         (df_train["image_path"].values, df_train["mask_path"].values)
    )
    ds_val = tf.data.Dataset.from_tensor_slices(
        (df_val["image_path"].values, df_val["mask_path"].values)
    )
    ds_test = tf.data.Dataset.from_tensor_slices(
        (df_test["image_path"].values, df_test["mask_path"].values)
    )

    return ds_train, ds_val, ds_test

ds_train, ds_val, ds_test = holdout(train_df)

In [46]:
def process_path(input_path, mask_path):
    """
    Load images from files.
    :input_path: the path to the satellite file
    :mask_path: the path to the mask file
    :return: The image and mask
    .. note:: Works with jpg images 
              Only the first channel is kept for the mask
    """
    
    IMAGE_SQ_SIZE = 224

    input_img = tf.io.read_file(input_path)   
    input_img = tf.io.decode_jpeg(input_img, channels=3)
    input_img =  tf.image.resize(input_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

    mask_img = tf.io.read_file(mask_path)   
    mask_img = tf.io.decode_jpeg(mask_img, channels=1)
    mask_img =  tf.image.resize(mask_img, [IMAGE_SQ_SIZE, IMAGE_SQ_SIZE])

   
    return input_img, mask_img

def normalize(image, mask):
    # image = tf.cast(image, tf.float32) / 255.

    return tf.math.divide(image, 255), tf.math.divide(mask, 255) 

In [47]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_train = ds_train.map(process_path) \
.map(normalize) \
.batch(batch_size=TEST_BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

In [51]:
ds_train

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None))>

In [52]:
ds_val = ds_val.map(process_path) \
.map(normalize) \
.batch(batch_size=TEST_BATCH_SIZE) \
.prefetch(buffer_size=AUTOTUNE)

TypeError: in user code:

    File "/tmp/ipykernel_1095180/2956518407.py", line 13, in process_path  *
        input_img = tf.io.read_file(input_path)

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.


In [53]:
# data.py

# model.py

In [54]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, ReLU
from tensorflow.keras.layers import BatchNormalization, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import plot_model

from tensorflow.keras.applications import VGG16




class SMR_Model():
    ''' creating our first lwb_smr models '''

    def __init__(self, input_shape):
        self.input_shape = input_shape

    def get_latest_model(self):
        model = self.build_vgg16_unet(self.input_shape)
        model = self.compile_model(model)

        return model

    def convolution_block(self, inputs, num_filters):
        ''' simple UNET convolution block with BatchNormalisation '''

        # convolution layer 1 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(inputs)  # padding='same' to avoid cut-down with conv
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # convolution layer 2 of the block
        x = Conv2D(num_filters, (3,3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # max pooling not used here as just the bridge

        return x

    def decoder_block(self, inputs, skip_tensor, num_filters):
        ''' decoder block for UNET '''
        # adds in the skips with concatenate
        x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same')(inputs) # stride important here to up-sample
        x = Concatenate()([x, skip_tensor])     # bringing in skip layer
        x = self.convolution_block(x, num_filters)

        return x

    def build_vgg16_unet(self, input_shape):
        ''' build vgg-16 '''

        inputs = Input(input_shape)

        # see actual VGG-16 here: https://github.com/keras-team/keras/blob/v2.9.0/keras/applications/vgg16.py#L43-L227
        vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
        # vgg16.summary()
        vgg16.trainable = False

        ''' Encoder - skip layers '''
        skip1 = vgg16.get_layer('block1_conv2').output #  256 x 256, 64 filters in vgg16
        skip2 = vgg16.get_layer('block2_conv2').output #  128 x 128, 128 filters in vgg16
        skip3 = vgg16.get_layer('block3_conv3').output #   64 x 64, 256 filters in vgg16
        skip4 = vgg16.get_layer('block4_conv3').output #   32 x 32, 512 filters in vgg16
        # display('skip4: ' + str(skip4.shape))

        # only need to specify the skip layers, as VGG16 is an Encoder
        # Therefore, VGG16 comes built with MaxPool2d, so we don't specify

        ''' Bridge '''
        bridge = vgg16.get_layer('block5_conv3').output # 16 x 16, with 512 filters in vgg16
        # display('bridge: ' + str(bridge.shape))


        ''' Decoder '''
        d1 = self.decoder_block(bridge, skip4, 512) #  512 filters, as per the bridge
        d2 = self.decoder_block(d1, skip3, 256) #  256 filters
        d3 = self.decoder_block(d2, skip2, 128) #  128 filters
        d4 = self.decoder_block(d3, skip1, 64)  #   64 filters

        ''' Output '''
        outputs = Conv2D(1, (1,1), padding='same', activation='sigmoid')(d4)

        model = Model(inputs, outputs, name='first_VGG16_UNET')

        return model

    def compile_model(self, m):
        ''' with accuracy, binaryIoU, AuC '''
        # binaryIoU metric
        threshold = 0.5
        binaryIoU = tf.keras.metrics.BinaryIoU(target_class_ids=[1], threshold=threshold)
        AuC = tf.keras.metrics.AUC()

        # Compile Model
        m.compile(
                    loss=LOSS,
                    optimizer='adam',
                    metrics=['accuracy', binaryIoU, AuC]
                    )
        return m


# utils.py

In [55]:
# mlflow
#
# and others...

import mlflow
from mlflow.tracking import MlflowClient
from memoized_property import memoized_property

MLFLOW_URI = "https://mlflow.lewagon.ai/"

class PushMLFlow():
    '''
        MLFLOW_URI = "https://mlflow.lewagon.ai/"
        EXPERIMENT_NAME = "[UK] [LONDON] [SOLAR_ROOF] TEST RUN" # template
        EXPERIMENT_TAGS = {
            'USER': 'test_user',
            'RUN NAME': 'test runs',
            'VERSION': '1.0.1',
            'DESCRIPTION': 'testing MLFlow Pipeline. Model - basic U-Net structure, 2 epochs, 15 images'
        }
    '''

    def __init__(self, experiment_name, experiment_tags):
        self.experiment_name = experiment_name
        self.experiment_tag = experiment_tags

        pass
    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id, tags=self.experiment_tag)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)


In [56]:
# trainer.py

In [57]:
# THIS NEEDS TO BE WRITTEN OUT PROPERLY!
# ESPECIALLY THE FIRST BIT THAT SHOULD BE IN DATA.py as Josh has been writing
import os

import tensorflow as tf

# from lwb_smr.CustomDataLoader import CustomDataLoader
# from lwb_smr.model import SMR_Model
# from lwb_smr.utils import PushMLFlow

from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.metrics import AUC, IoU


class Trainer():
    def __init__(self):
        pass

    def set_model(self, loss=LOSS):
        self.loss = loss
        # Instantiate Model
        # our_input_shape = (224,224,3)

        getVGG16 = SMR_Model(UNET_INPUT_SHAPE)
        self.model = getVGG16.get_latest_model()
        # see compile in SMR_Model

    def start_mlflow(self):
        p = PushMLFlow(EXPERIMENT_NAME, EXPERIMENT_TAGS)
        return p.mlflow_run

    def run(self):

        print(80*'-')
        print('------SETTING FOR DATA RUN------')

        ### 
        ##
        #
        # customdata = self.just_get_the_data_loaded()

        print(80*'-')
        print('------MODEL RUNNING------')

        # set mflow
        self.MFLOW = self.start_mlflow()

        # set model
        self.set_model()

        mc = ModelCheckpoint(f'{CHECKPOINT_PATH}/{CHECKPOINT_NAME}.h5', save_best_only=True) # could put path here
        es = EarlyStopping(patience=5, restore_best_weights=True)
        self.model.fit(
            ds_train,
            validation_data=ds_val,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            callbacks=[mc, es]
            )
        
        ## This worked...
        # self.model.fit(
        #     customdata,
        #     # validation_split=0.3,
        #     batch_size=BATCH_SIZE,
        #     epochs=EPOCHS,
        #     callbacks=[mc, es]
        #     )
    

        model_path_and_filename = f'{MODEL_PATH}/{MODEL_NAME}/.h5'
        # '../models/220610_sampleCHECKER_UNET_input_shape_224x224x3.h5'
        self.model.save(model_path_and_filename)
        # HERE HERE HERE HERE
        # line above missing: self.
        
        self.MFLOW.mlflow_log_param('loss', self.loss)

        print(80*'=')
        print('------MODEL RUN SUCCESFULLY COMPLETED------')

        self.evaluate()

    def evaluate(self):
        print(80*'-')
        print('------MODEL EVALUATING------')
        results = self.model.evaluate(self.X_test, self.y_test)
        self.MFLOW.mlflow_log_metric('loss', results)
        print(80*'=')
        print('------MODEL EVALUATED------')

if __name__ == '__main__':
    pass
    t = Trainer()
    t.run()

--------------------------------------------------------------------------------
------SETTING FOR DATA RUN------
--------------------------------------------------------------------------------
------MODEL RUNNING------
Epoch 1/2


2022-06-10 11:42:13.794296: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: ../../raw_data/train_mask_tiles_jpeg/vienna32_x08_y08.jpeg; No such file or directory
2022-06-10 11:42:13.794870: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: ../../raw_data/train_mask_tiles_jpeg/austin31_x01_y12.jpeg; No such file or directory
2022-06-10 11:42:13.794925: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: ../../raw_data/train_mask_tiles_jpeg/chicago11_x01_y11.jpeg; No such file or directory
2022-06-10 11:42:13.794960: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file_read_ops.cc:114 : NOT_FOUND: ../../raw_data/train_mask_tiles_jpeg/tyrol-w3_x14_y11.jpeg; No such file or directory
2022-06-10 11:42:13.795013: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at whole_file

NotFoundError: Graph execution error:

../../raw_data/train_mask_tiles_jpeg/vienna32_x08_y08.jpeg; No such file or directory
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_14129]

In [ ]:
########
####
###
#
#        binaryIoU goes up with tensorslices, and not with customdata
#
##
###
####
########

In [ ]:
# @@@@@@@@@@@@@ but will it go to 2+ epochs??

In [ ]:
# YES, finally...!

In [ ]:
# BUT!